In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

# 1. **Data Preparation**
# Dataset Loading & Preprocessing
transform = transforms.Compose([
    transforms.RandomRotation(20),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet stats
])

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# Dataset Splitting (80% train, 20% validation)
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# DataLoader Setup
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# 2. **Model Initialization**
# Initialize models (ResNet18, MobileNetV2, and AlexNet)
models = {
    "ResNet18": torchvision.models.resnet18(pretrained=True),
    "MobileNetV2": torchvision.models.mobilenet_v2(pretrained=True),
    "AlexNet": torchvision.models.alexnet(pretrained=True)
}

# Modify the final layer for CIFAR-10 (10 output classes)
models["ResNet18"].fc = nn.Linear(models["ResNet18"].fc.in_features, 10)
models["MobileNetV2"].classifier[1] = nn.Linear(models["MobileNetV2"].classifier[1].in_features, 10)
models["AlexNet"].classifier[6] = nn.Linear(models["AlexNet"].classifier[6].in_features, 10)

# Move models to GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
for model in models.values():
    model.to(device)

# 3. **Model Training and Validation**
train_losses = {"ResNet18": [], "MobileNetV2": [], "AlexNet": []}
val_losses = {"ResNet18": [], "MobileNetV2": [], "AlexNet": []}
val_accuracies = {"ResNet18": [], "MobileNetV2": [], "AlexNet": []}
epochs = 10
criterion = nn.CrossEntropyLoss()

# Train each model
for model_name, model in models.items():
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for epoch in range(epochs):
        model.train()
        running_train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        avg_train_loss = running_train_loss / len(train_loader)
        train_losses[model_name].append(avg_train_loss)

        # Validation
        model.eval()
        running_val_loss = 0.0
        correct, total = 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                running_val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        avg_val_loss = running_val_loss / len(val_loader)
        val_losses[model_name].append(avg_val_loss)
        val_accuracy = correct / total
        val_accuracies[model_name].append(val_accuracy)

        print(f"{model_name} Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# 4. **Confusion Matrix Visualization for each model**
for model_name, model in models.items():
    model.eval()
    all_labels = []
    all_preds = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.cpu().numpy())
            all_preds.extend(predicted.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(cmap='Blues')
    plt.title(f"Confusion Matrix - {model_name}")
    plt.show()

# 5. **Loss and Accuracy Plots**
plt.figure(figsize=(15, 6))

# Training and Validation Loss
plt.subplot(1, 2, 1)
for model_name in models.keys():
    plt.plot(train_losses[model_name], label=f'{model_name} Train Loss')
    plt.plot(val_losses[model_name], label=f'{model_name} Val Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Training and Validation Loss')

# Validation Accuracy
plt.subplot(1, 2, 2)
for model_name in models.keys():
    plt.plot(val_accuracies[model_name], label=f'{model_name} Val Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Validation Accuracy')

plt.tight_layout()
plt.show()

# I got the execution at first runtime with GPU on GoogleColab with output of Validation Accuracys but failed to put images of confusion matrix and graphs in research paper as I re=executed it and then colabs GPU consumption was exceeded for free trial.